In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [4]:
startYear = 2020
startMonth =1
startDay = 1
endYear = 2022
endMonth = 3
endDay = 1

In [12]:
 aoi = ee.Geometry.Polygon([[[
            35.008243,
            10.522199
        ],
            [
            35.008243,
            11.266588
        ],
            [
            35.387092,
            11.266588
        ],
            [
            35.387092,
            10.522199
        ],
            [
            35.008243,
            10.522199
        ]]])

In [22]:
SAR = S1_SAR_col(startYear,startMonth,startDay,endYear,endMonth,endDay)

In [27]:
url = SAR_timeseries_url(SAR,aoi)

In [28]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/bb8572c6be0e469d7cd0df876b9ab2de-afd38c7651bf944d592c32443d39ce01:getPixels
